# Identifying Potential Customers Based on Reddit Posts 

#### By: Julia Kelman: [GitHub](https://git.generalassemb.ly/julia-kelman/)

## Problem Statement

### Loading Libraries 

In [1]:
import pandas as pd
import datetime as dt
import time
import requests

### Data Gathering Function

In [2]:
def query_pushshift(subreddit, kind = 'submission', day_window = 30, n = 5):
    SUBFIELDS = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']
    
    # establish base url and stem
    BASE_URL = f"https://api.pushshift.io/reddit/search/{kind}"  
    stem = f"{BASE_URL}?subreddit={subreddit}&size=500" 
    
    # instantiate empty list for temp storage
    posts = []
    
    # implement for loop with `time.sleep(2)`
    for i in range(1, n + 1):
        URL = "{}&after={}d".format(stem, day_window * i)
        print("Querying from: " + URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        posts.append(df)
        time.sleep(2)
    
    # pd.concat storage list
    full = pd.concat(posts, sort=False)
    
    # if submission
    if kind == "submission":
        # select desired columns
        full = full[SUBFIELDS]
        # drop duplicates
        full.drop_duplicates(inplace = True)
        # select `is_self` == True
        full = full.loc[full['is_self'] == True]

    # create `timestamp` column
    full['timestamp'] = full["created_utc"].map(dt.date.fromtimestamp)
    
    print("Query Complete!")    
    return full 

# Function created by Mahdi Shadkam-Farrokhi

### OCD Reddit Data 

In [3]:
# Gathering Data 
ocd_results = query_pushshift("OCD")

Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=OCD&size=500&after=30d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=OCD&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=OCD&size=500&after=90d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=OCD&size=500&after=120d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=OCD&size=500&after=150d
Query Complete!


In [4]:
# Checking how many observations we have 
ocd_results.shape

(2209, 9)

In [5]:
ocd_results.head()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
0,Was convinced to try an outpatient treatment p...,[removed],OCD,1584570256,stringthanks,0,1,True,2020-03-18
1,To anyone who has contamination ocd: what are ...,,OCD,1584570641,Mental_senstrias,9,1,True,2020-03-18
2,Sertraline HCL (Generic Zoloft) w/out Insurance?,"Amid the coronavirus economic recession, there...",OCD,1584570720,BigOof673,2,2,True,2020-03-18
3,Feeling like going mad,"I suffer from severe OCD, related to magic thi...",OCD,1584570825,Bobby_Crazykite,4,3,True,2020-03-18
4,Worried about this sticky adhesive,So I noticed that the little circles on cabine...,OCD,1584570850,JakeHassle,0,1,True,2020-03-18


### ASD Reddit Data

In [6]:
# Gathering ASD Data 
asd_results = query_pushshift("autism")

Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=autism&size=500&after=30d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=autism&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=autism&size=500&after=90d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=autism&size=500&after=120d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=autism&size=500&after=150d
Query Complete!


In [7]:
# Checking how many observations we have 
asd_results.shape

(1653, 9)

In [8]:
asd_results.head()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
1,Lets calm down on the hat thing and have some ...,Please,autism,1584572407,Yormankaaz,10,1,True,2020-03-18
2,I’m not sure if this the best place to ask but...,At the moment I (f18) have no children but whe...,autism,1584572497,oorangecucumber,17,1,True,2020-03-18
4,I feel the need to disclose I'm autistic on dates,"I realise I feel this way, and I feel complica...",autism,1584574184,graveyard_flirt,8,1,True,2020-03-18
8,Something that bothers me (rant),I really dislike that people sometimes assume ...,autism,1584580960,Olliewildlife401,5,1,True,2020-03-18
9,My mom and her boyfriend won't let me take my ...,My mom and her boyfriend are very against medi...,autism,1584582027,Sledm0r,17,1,True,2020-03-18


### Combining and Saving Data

In [9]:
data = pd.concat([ocd_results, asd_results])

In [10]:
data.shape

(3862, 9)

In [12]:
data.to_csv("./data/data.csv", index = False)

## References

[Autism Reddit](https://www.reddit.com/r/autism/)  
[OCD Reddit](https://www.reddit.com/r/OCD/)